In [ ]:
PPMC_ORDER = 4

import csv
from PPMC import PPMC

text = "i got them in a very timely manner and they're all very large and green. there seems to be an inordinate amount of the seeds e out a weird beige color, but it still tastes great and is a much better deal than any of the indian markets in town. cheers to frontier."

trainRDD = sc.textFile("train_sin_repeticiones.csv").mapPartitions(lambda x: csv.reader(x))
trainRDD = trainRDD.filter(lambda line: line[ID] != "Id")

#se podria hacer un map antes que limpie el texto
#comprimo todo el train con rdd
trainRDD = trainRDD.map(lambda line: (line[PREDICTION], (len(PPMC(PPMC_ORDER).compress(line[TEXT])), 1))

#comprimo el texto de prueba con los modelos de todo el rdd
trainRDD = trainRDD.map(lambda line: (line[0], (line[1][0].compress(text), line[1][1])))
                        
trainRDD = trainRDD.reduceByKey(lambda value1, value2: (value1[0]+value2[0], value1[1]+value2[1]))

trainRDD = trainRDD.filter(lambda line1, line2: line1 if line1[1][0]/line1[1][1] < line2[1][0]/line2[1][1] else line2)